In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import SimpleITK as sitk 
import os
from tqdm.notebook import tqdm, trange
from skimage.transform import downscale_local_mean
from typing import List, Tuple, Any, Dict, Set, Optional
from sklearn.model_selection import train_test_split
os.environ["PATH"] = "/Users/ivannovikov/Downloads/elastix-5.0.0-mac/bin" + os.pathsep + os.environ["PATH"]
os.environ["DYLD_LIBRARY_PATH"] = "/Users/ivannovikov/Downloads/elastix-5.0.0-mac/lib"
import elastix
from scrollview import ScrollView
from misc import Config
from model import *
from operator import itemgetter
%matplotlib qt

In [2]:
ELASTIX_PATH = os.path.join(r'/Users/ivannovikov/Downloads/elastix-5.0.0-mac/bin/elastix')
TRANSFORMIX_PATH = os.path.join(r'/Users/ivannovikov/Downloads/elastix-5.0.0-mac/bin/transformix')
FOLDER_RAW = r'/Users/ivannovikov/Documents/CSMIA/TrainingData/'
FOLDER_PREPROCESSED = r'/Users/ivannovikov/Documents/CSMIA/DataPreprocessed'
FOLDER_PARAMETERS = r'/Users/ivannovikov/Documents/CSMIA/parameters'
FOLDER_RESULTS = r'/Users/ivannovikov/Documents/CSMIA/results'

In [3]:
config = Config(
    folder_raw = FOLDER_RAW, 
    folder_preprocessed = FOLDER_PREPROCESSED,
    folder_parameters = FOLDER_PARAMETERS,
    folder_results = FOLDER_RESULTS,
    seed = 0,
    train_size = 12,
)

In [4]:
model = Model(
    config = config,
    elastix_path = ELASTIX_PATH,
    transformix_path = TRANSFORMIX_PATH,
)

In [5]:
model.load()

Loading Data:   0%|          | 0/15 [00:00<?, ?it/s]

In [6]:
model.preprecess_and_save(downscale_factor=4, normalize="none")

Saving Preprocessed Data:   0%|          | 0/15 [00:00<?, ?it/s]

In [7]:
model.run_registration()

Running with parameters BSpline.txt


  0%|          | 0/3 [00:00<?, ?it/s]

/Users/ivannovikov/Downloads/elastix-5.0.0-mac/bin/elastix -f /Users/ivannovikov/Documents/CSMIA/DataPreprocessed/p107/mr_bffe.mhd -m /Users/ivannovikov/Documents/CSMIA/DataPreprocessed/p125/mr_bffe.mhd -p /Users/ivannovikov/Documents/CSMIA/parameters/BSpline.txt -out /Users/ivannovikov/Documents/CSMIA/results/BSpline/p107_as_fixed/p125_as_moving
/Users/ivannovikov/Downloads/elastix-5.0.0-mac/bin/elastix -f /Users/ivannovikov/Documents/CSMIA/DataPreprocessed/p107/mr_bffe.mhd -m /Users/ivannovikov/Documents/CSMIA/DataPreprocessed/p135/mr_bffe.mhd -p /Users/ivannovikov/Documents/CSMIA/parameters/BSpline.txt -out /Users/ivannovikov/Documents/CSMIA/results/BSpline/p107_as_fixed/p135_as_moving
/Users/ivannovikov/Downloads/elastix-5.0.0-mac/bin/elastix -f /Users/ivannovikov/Documents/CSMIA/DataPreprocessed/p107/mr_bffe.mhd -m /Users/ivannovikov/Documents/CSMIA/DataPreprocessed/p115/mr_bffe.mhd -p /Users/ivannovikov/Documents/CSMIA/parameters/BSpline.txt -out /Users/ivannovikov/Documents/CSMI

In [8]:
model.threshold_staple = 0.95
model.run_staple()

Running for parameters BSpline.txt


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
model.scores

[[0.6653657401941012, 0.5464058137942353, 0.7694382923219628]]

In [67]:
dice_score(model.segmentations[0][0]>0.99999999999, model.ground_truths[0], eps=1e-5)

0.7432233910951942

In [9]:
im = sitk.GetArrayFromImage(sitk.ReadImage(FOLDER_PREPROCESSED + "/p108/mr_bffe.mhd"))
mask = sitk.GetArrayFromImage(sitk.ReadImage(FOLDER_PREPROCESSED + "/p108/prostaat.mhd"))

registered = sitk.GetArrayFromImage(sitk.ReadImage(FOLDER_RESULTS + "/BSpline/p108_as_fixed/p102_as_moving/result.0.mhd"))
transformed = sitk.GetArrayFromImage(sitk.ReadImage(FOLDER_RESULTS + "/BSpline/p108_as_fixed/p102_as_moving/result.mhd"))

In [42]:
view = lambda x, ax : ScrollView(x).plot(ax, vmin=x.min(), vmax=x.max())

In [12]:
a = sitk.GetArrayFromImage(res[0]) > 0.95
fig, ax = plt.subplots(1,1)
view(a, ax)